In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K


Using TensorFlow backend.


In [6]:
# dimensions of our images.
input_shape=(150,150,3)
img_width, img_height = 150, 150

train_data_dir = '/content/drive/My Drive/train'
validation_data_dir = '/content/drive/My Drive/test'
nb_train_samples = 319
nb_validation_samples = 24
epochs = 10
batch_size = 16

In [7]:
#rescaling data
datagen=ImageDataGenerator(rescale=1./255)

train_generator=datagen.flow_from_directory(
         train_data_dir,
         target_size=(img_width,img_height),
         batch_size=8,
         class_mode='categorical')

validation_generator=datagen.flow_from_directory(
         validation_data_dir,
         target_size=(img_width,img_height),
         batch_size=16,
         class_mode='categorical')


Found 319 images belonging to 3 classes.
Found 24 images belonging to 3 classes.


In [8]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))


In [9]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 64)       

In [15]:
model.compile(loss='binary_crossentropy', # or categorical_crossentropy
              optimizer='adam',# or adagrad
              metrics=['accuracy'])


In [16]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [17]:
# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

print(train_generator.class_indices)

Found 319 images belonging to 3 classes.
{'jeans': 0, 'sarees': 1, 'trousers': 2}


In [18]:
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 24 images belonging to 3 classes.


In [19]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)


Epoch 1/25
19/19 [==============================] - 3s 156ms/step - loss: -424089.7271 - accuracy: 0.3564 - val_loss: -685744.1250 - val_accuracy: 0.4375
Epoch 2/25
19/19 [==============================] - 3s 142ms/step - loss: -430611.7835 - accuracy: 0.4572 - val_loss: -1524594.2500 - val_accuracy: 0.3750
Epoch 3/25
19/19 [==============================] - 3s 145ms/step - loss: -577221.8367 - accuracy: 0.3808 - val_loss: -996224.3750 - val_accuracy: 0.4375
Epoch 4/25
19/19 [==============================] - 3s 146ms/step - loss: -594266.1521 - accuracy: 0.4013 - val_loss: -1961302.0000 - val_accuracy: 0.3750
Epoch 5/25
19/19 [==============================] - 3s 149ms/step - loss: -989147.0950 - accuracy: 0.3642 - val_loss: -3511791.2500 - val_accuracy: 0.2500
Epoch 6/25
19/19 [==============================] - 3s 146ms/step - loss: -1018741.3758 - accuracy: 0.4178 - val_loss: 356809.2188 - val_accuracy: 0.6250
Epoch 7/25
19/19 [==============================] - 3s 144ms/step - loss:

In [20]:
model.save('model1.h5')

#Prediction 

In [21]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
from os import listdir
from os.path import isfile, join


In [22]:
# dimensions of our images
img_width, img_height = 150, 150

In [24]:
# load the model we saved
model = load_model('model1.h5')
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [28]:
mypath = "/content/drive/My Drive/Predict/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(onlyfiles)
# predicting images
jeans_counter = 0 
trousers_counter  = 0
sarees_counter = 0
for file in onlyfiles:
    img = image.load_img(mypath+file, target_size=(img_width, img_height))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    
    images = np.vstack([x])
    classes = model.predict_classes(images, batch_size=10)
    classes = classes[0][0]
    
    if classes == 0:
        print(file + ": " + 'jeans')
        jeans_counter += 1


    if classes == 1:
        print(file + ": " + 'sarees')
        sarees_counter += 1

    else:
        print(file + ": " + 'trousers')
        sarees_counter += 1
print("Total jeans :",jeans_counter)
print("Total sarees :",sarees_counter)
print("Total trousers :",trousers_counter)

['83.jpg', '226.jpg', '213.jpg', '221.jpg', '47.jpg', '35.jpg', '69.jpg', '54.jpg', '38.jpg', '78.jpg', '105.jpg', '108.jpg', '7.jpg', '485.jpg', '382.jpg', '432.jpg', '433.jpg', '396.jpg', '430.jpg', '473.jpg', '487.jpg', '386.jpg', '371.jpg']
83.jpg: sarees
226.jpg: sarees
213.jpg: sarees
221.jpg: sarees
47.jpg: sarees
35.jpg: sarees
69.jpg: sarees
54.jpg: sarees
38.jpg: sarees
78.jpg: sarees
105.jpg: sarees
108.jpg: sarees
7.jpg: sarees
485.jpg: sarees
382.jpg: sarees
432.jpg: sarees
433.jpg: sarees
396.jpg: sarees
430.jpg: sarees
473.jpg: sarees
487.jpg: sarees
386.jpg: sarees
371.jpg: sarees
Total jeans : 0
Total sarees : 23
Total trousers : 0
